# 05. Обучение CNN

## План
1. CIFAR10: baseline
2. Просто добавим аугментаций
3. Pretrained vs from scratch
4. LR scheduling

## 1. CIFAR10: baseline

Мы уже умеем составлять несложные архитектуры нейросетей и обучать их на произвольных (картиночных) датасетах.
На этом семинаре мы поговорим о том, какие ручки можно покрутить, чтобы улучшать результаты.

Перечислим некоторые (но точно не все) из таких ручек:

* Параметры модели
  * тип архитектуры
    * семейство (ResNet / EfficientNet / ...)
    * размер модели (ResNet18 / ResNet101?)
  * число обучаемых слоёв
    * warmup при дообучении
  * ...

* Параметры оптимизации
  * собственно оптимизатор (SGD / Adam + вариации / ...)
  * learning rate
    * scheduling
  * momentum
  * weight decay
  * ...
  
* Параметры данных
  * веса классов / сэмплирование
  * набор и сила аугментаций
  * добавление / чистках
  * ...

* Параметры обучения
  * размер батча
  * функция потерь
  * целевая метрика (да, не `val_loss`-ом единым)  
  * критерий остановки
  * ...
  
* ...


Но решение любой задачи начинается с построения бейзлайна!

### 1.1. Получаем данные

In [ ]:
import os
import glob
import pickle
import tqdm
import cv2

Если еще не скачивали данные:

Если уже скачали:

In [ ]:
print(len(filenames), len(labels))

In [ ]:
cifar10_class_map = {
    0: "airplane",
    1: "automobile",
    2: "bird",
    3: "cat", 
    4: "deer",
    5: "dog",
    6: "frog", 
    7: "horse",
    8: "ship",
    9: "truck"
}

### 1.2. Собираем датасет и знакомимся с данными

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T

In [ ]:
class CIFAR10Dataset(Dataset):

    def __init__(self, filenames, labels, split, transforms):
        self.filenames = filenames
        self.labels = labels
        self.split = split
        self.transforms = transforms

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, i):
        image = cv2.imread(self.filenames[i])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32) / 255.
        
        # v NOTE THIS v
        image_tensor = self.transforms(image)
        # ^ NOTE THIS ^

        label = self.labels[i]
        
        return image_tensor, label

    @staticmethod
    def collate_fn(items):
        images_batch = torch.zeros(len(items), 3, 32, 32)
        labels_batch = torch.zeros(len(items))
        for i, item in enumerate(items):
            images_batch[i] = item[0]
            labels_batch[i] = item[1]
        return images_batch.float(), labels_batch.long()

Прежде мы при обращении к картинкам через датасет обрабатывали их руками (конвертировали в тензор, например).
Можно (и вообще говоря нужно) делать это через механизм трансформаций:

In [ ]:
transforms_simple = T.Compose([
    T.ToTensor(),
    T.Normalize([0.5, 0.5, 0.5], [0.25, 0.25, 0.25])  # should've computed this on train data, but...
])

Здесь мы использовали для трансформаций модуль `torhvision.transforms`, но есть и альтернативы (о них чуть ниже).
Пока что мы ограничились включением в трансформации только базовых операций - конвертации в тензор и нормализации.

In [ ]:
filenames_train, filenames_val, labels_train,  labels_val = train_test_split(filenames, labels, train_size=0.9, stratify=labels)

In [ ]:
dataset_train = CIFAR10Dataset(filenames_train, labels_train, "train", transforms_simple)

Для обратной конвертации в картинку (чтобы отрисовать ее, например), нужно сделать "де-нормализацию":

**Задача**: реализовать функцию `tensor_to_image`, получающую на вход нормализованный тензор `(3, h, w)`, возвращающую де-нормализованный массив `(h, w, 3)`.

In [ ]:
def tensor_to_image(tensor):

    ### YOUR CODE HERE

    ### END OF YOUR CODE

    return image

In [ ]:
image = np.random.uniform(size=(32, 32, 3))
tensor = (torch.from_numpy(image).permute(2, 0, 1) - 0.5) / 0.25

np.testing.assert_array_equal(image, tensor_to_image(tensor))

Посмотрим глазами на данные:

In [ ]:
indexes_to_show = np.random.choice(len(dataset_train), size=64, replace=False)

plt.figure(figsize=(18, 14))
for i, index in enumerate(indexes_to_show):
    tensor, label = dataset_train[index]
    image = tensor_to_image(tensor)
    plt.subplot(8, 8, i + 1)
    plt.imshow(image)
    plt.axis(False)
    plt.title(f"GT: {label} ({cifar10_class_map[label]})")
plt.show()

Обычно полезно провести разведывательный анализ данных (EDA).
Сейчас ограничимся тем, что посмотрим на распределение количества картинок по классам.

**Задача**: любым удобным способ вывести количество изображений по каждому классу в обучающем датасете.

In [ ]:
### YOUR CODE HERE

### END OF YOUR CODE

Не забудем собрать валидационный датасет, и двинемся дальше:

In [ ]:
dataset_val = CIFAR10Dataset(filenames_val, labels_val, "val", transforms_simple)

### 1.3. Собираем модель

Начнем, как и собирались, с бейзлайна.
Бейзлайн - это какое-то простое решение, которые конкретно вы можете быстро реализовать и проверить.
Чуть позже бейзлайном вы будете считать уже ResNet34, но пока напишем его ручками.

conv -> bn -> relu -> conv -> bn -> relu (pool)

**Задание**: реализуйте метод для инициализации блока сверточной сети. Блок должен работать так:
* conv 3x3 / in_channels -> out_channels
* batchnorm2d
* relu
* conv 3x3
* batchnorm2d
* relu
* (optionally) maxpool 2x2

In [ ]:
class CNNBlock(nn.Module):

    def __init__(self, in_channels, out_channels, pool=True):
        super(CNNBlock, self).__init__()

        ### YOUR CODE HERE

        ### END OF YOUR CODE

        
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        return x

Соберем из этих блоков сеть:

In [ ]:
cnn_baseline = nn.Sequential(
    CNNBlock(3, 32),
    CNNBlock(32, 64),
    CNNBlock(64, 128),
    CNNBlock(128, 256),
    CNNBlock(256, 512),
    
    # v NOTE THIS
    nn.AdaptiveAvgPool2d((1, 1)),
    # ^ NOTE THIS ^
    
    nn.Flatten(),
    nn.Linear(512, 10)
)

In [ ]:
x = torch.randn(4, 3, 32, 32)
y = cnn_baseline(x)
y.shape

### 1.4. Учим

In [ ]:
num_epochs = 8
batch_size = 128
lr = 3e-4

device = torch.device("cuda:7") if torch.cuda.is_available() else torch.device("cpu")
print(device)

In [ ]:
dataloader_train = DataLoader(dataset_train, 
                              collate_fn=CIFAR10Dataset.collate_fn, 
                              batch_size=batch_size, shuffle=True, drop_last=True, 
                              num_workers=4, pin_memory=True)

dataloader_val = DataLoader(dataset_val, 
                            collate_fn=CIFAR10Dataset.collate_fn, 
                            batch_size=batch_size, shuffle=False, drop_last=False, 
                            num_workers=4, pin_memory=True)

Инициализируйте сами необходимый лосс и оптимизатор Adam, взяв готовые из `pytorch`:

In [ ]:
### YOUR CODE HERE

# loss_fn = ...

### END OF YOUR CODE

In [ ]:
### YOUR CODE HERE

# optimizer = ...

### END OF YOUR CODE

Раньше у нас были отдельные методы для обучения/валидации - теперь мы готовы сделать из них один:

In [ ]:
def run_epoch(stage, model, dataloader, loss_fn, optimizer, epoch, device):
    
    # v NOTE THIS v
    if stage == "train":
        model.train()
        torch.set_grad_enabled(True)
    else:
        torch.set_grad_enabled(False)
        model.eval()
    # ^ NOTE THIS ^

    model = model.to(device)
    
    losses = []
    for batch in tqdm.tqdm(dataloader, total=len(dataloader), desc=f"epoch: {str(epoch).zfill(3)} | {stage:5}"):
        xs, ys_true = batch
                
        ys_pred = model(xs.to(device))
        loss = loss_fn(ys_pred, ys_true.to(device))

        if stage == "train":
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
                
        losses.append(loss.detach().cpu().item())

    return np.mean(losses)

Кроме того, мы готовы к чему-то большему, чем просто брать последний чекпоинт модели.
Будем контролировать значение целевой метрики (сегодня это `val_loss`), и сохранять чекпоинт модели в случае, если он лучший.

Для этого вспомните-ка, как сохранять и загружать веса моделей.

**Задание**: реализуйте функции `save_checkpoint()` & `load_checkpoint()`. На входе объект модели и имя файла, на выходе - ничего (но в случае загрузки модель должна получить новые веса).

In [ ]:
def save_checkpoint(model, filename):

    ### YOUR CODE HERE

    ### END OF YOUR CODE


def load_checkpoint(model, filename):

    ### YOUR CODE HERE

    ### END OF YOUR CODE

In [ ]:
my_model = nn.Linear(100, 1)
my_model.weight *= 1e6
save_checkpoint(my_model, "test.pth.tar")

my_model_new = nn.Linear(100, 1)
load_checkpoint(my_model_new, "test.pth.tar")

torch.testing.assert_allclose(my_model.weight, my_model_new.weight)

Экспериментов у нас будет много, поэтому для экономии кода обернем все, что нужно для обучения, в функцию `run_experiment()`:

In [ ]:
def run_experiment(model, dataloader_train, dataloader_val, loss_fn, optimizer, num_epochs, device, output_dir):
    
    train_losses = []
    val_losses = []

    best_val_loss = np.inf
    best_val_loss_epoch = -1
    best_val_loss_fn = None

    os.makedirs(output_dir, exist_ok=True)

    for epoch in range(num_epochs):
        train_loss = run_epoch("train", model, dataloader_train, loss_fn, optimizer, epoch, device)
        train_losses.append(train_loss)

        val_loss = run_epoch("val", model, dataloader_val, loss_fn, optimizer, epoch, device)
        val_losses.append(val_loss)

        print(f"epoch: {str(epoch).zfill(3)} | train_loss: {train_loss:5.3f}, val_loss: {val_loss:5.3f} (best: {best_val_loss:5.3f})")

        if val_loss < best_val_loss:

            best_val_loss = val_loss
            best_val_loss_epoch = epoch

            output_fn = os.path.join(output_dir, f"epoch={str(epoch).zfill(2)}_valloss={best_val_loss:.3f}.pth.tar")
            save_checkpoint(model, output_fn)
            print(f"New checkpoint saved to {output_fn}")

            best_val_loss_fn = output_fn

        print()

    print (f"Best val_loss = {best_val_loss:.3f} reached at epoch {best_val_loss_epoch}")
    load_checkpoint(model, best_val_loss_fn)

    return train_losses, val_losses, best_val_loss, model

Запустим:

In [ ]:
train_losses_baseline, val_losses_baseline, best_val_loss_baseline, cnn_baseline = run_experiment(
    cnn_baseline, dataloader_train, dataloader_val, loss_fn, optimizer, num_epochs, device, "checkpoints_baseline"
)

Смотрим результаты:

In [ ]:
def plot_losses(train_losses, val_losses, title):
    plt.figure(figsize=(12, 5))
    plt.title(title)
    plt.plot(train_losses, label="train")
    plt.plot(val_losses, label="val")
    plt.xlabel("epoch")
    plt.ylabel("loss")
    plt.grid(True)
    plt.legend()
    plt.show()

In [ ]:
plot_losses(train_losses_baseline, val_losses_baseline, title="cnn_baseline")

Считаем метрики:

In [ ]:
def collect_predictions(model, dataloader, device):
    model.eval()
    model = model.to(device)
    torch.set_grad_enabled(False)

    labels_all = []
    probs_all = []
    preds_all = []
    for batch in tqdm.tqdm(dataloader, total=len(dataloader)):
        images, labels = batch

        logits = model(images.to(device)).cpu()
        probs = logits.softmax(dim=1)
        max_prob, max_prob_index = torch.max(probs, dim=1)

        labels_all.extend(labels.numpy().tolist())
        probs_all.extend(max_prob.numpy().tolist())
        preds_all.extend(max_prob_index.numpy().tolist())
    
    return labels_all, probs_all, preds_all

In [ ]:
train_labels, train_probs, train_preds = collect_predictions(cnn_baseline, dataloader_train, device)

accuracy_train = accuracy_score(train_labels, train_preds)
accuracy_train

In [ ]:
train_labels[:5], train_preds[:5], train_probs[:5]

In [ ]:
val_labels, val_probs, val_preds = collect_predictions(cnn_baseline, dataloader_val, device)

accuracy_val = accuracy_score(val_labels, val_preds)
accuracy_val

## 2. Просто добавим аугментаций

Одна из базовых вещей при обучении - это аугментации. Можно делать с помощью `torchvision.transforms`, а можно взять сторонние библиотеки - например, [`albumentations`](https://albumentations.ai/). Есть и [более необычные вещи](https://pytorch.org/vision/main/generated/torchvision.transforms.AutoAugment.html), но о них мы отдельно говорить не будем.

С аугментациями можно переборщить, поэтому начнем с малого:

In [ ]:
transforms_aug = T.Compose([
    T.ToTensor(),
    T.RandomHorizontalFlip(),
    T.RandomRotation(degrees=15),
    T.Normalize([0.5, 0.5, 0.5], [0.25, 0.25, 0.25])
])

In [ ]:
dataset_aug_train = CIFAR10Dataset(filenames_train, labels_train, "train", transforms_aug)

In [ ]:
indexes_to_show = np.random.choice(len(dataset_aug_train), size=64, replace=False)

plt.figure(figsize=(18, 14))
for i, index in enumerate(indexes_to_show):
    tensor, label = dataset_aug_train[index]
    image = tensor_to_image(tensor)
    plt.subplot(8, 8, i + 1)
    plt.imshow(image)
    plt.axis(False)
    plt.title(f"GT: {label} ({cifar10_class_map[label]})")
plt.show()

Посмотрим на динамику обучения с аугментациями (обратите внимание, валидационный датасет остался прежним):

In [ ]:
cnn_aug = nn.Sequential(
    CNNBlock(3, 32),
    CNNBlock(32, 64),
    CNNBlock(64, 128),
    nn.AdaptiveAvgPool2d((1, 1)),
    nn.Flatten(),
    nn.Linear(128, 10)
)

Увеличим число эпох (забегая вперед - переобучение мы немного снизим, поэтому имеет смысл добавить итераций).

In [ ]:
num_epochs = 32
batch_size = 128
lr = 3e-4

device = torch.device("cuda:7") if torch.cuda.is_available() else torch.device("cpu")
print(device)

In [ ]:
dataloader_aug_train = DataLoader(dataset_aug_train, 
                                  collate_fn=CIFAR10Dataset.collate_fn, 
                                  batch_size=batch_size, shuffle=True, drop_last=True, 
                                  num_workers=4, pin_memory=True)

dataloader_val = DataLoader(dataset_val, 
                            collate_fn=CIFAR10Dataset.collate_fn, 
                            batch_size=batch_size, shuffle=False, drop_last=False, 
                            num_workers=4, pin_memory=True)

In [ ]:
loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(cnn_aug.parameters(), lr=lr)

In [ ]:
train_losses_aug, val_losses_aug, best_val_loss_aug, cnn_aug = run_experiment(
    cnn_aug, dataloader_aug_train, dataloader_val, loss_fn, optimizer, num_epochs, device, "checkpoints_aug"
)

In [ ]:
plot_losses(train_losses_aug, val_losses_aug, title="cnn_aug")

In [ ]:
train_labels, train_probs, train_preds = collect_predictions(cnn_aug, dataloader_aug_train, device)

accuracy_train = accuracy_score(train_labels, train_preds)
accuracy_train

In [ ]:
val_labels, val_probs, val_preds = collect_predictions(cnn_aug, dataloader_val, device)

accuracy_val = accuracy_score(val_labels, val_preds)
accuracy_val

## 3. Pretrained & from scratch

Важнейший прием, которым следует овладеть - это пользоваться готовыми моделями :)

Часто (*да почти всегда*) лучше учиться не со случайных весов. Если у вас в наличии есть модель, уже обученная на хоть сколько-нибудь смежном домене с целевым - надо брать и дообучаться с нее.

Откуда брать модели?
* [`torchvision.models`](https://pytorch.org/vision/0.8/models.html)
* [`pytorch_image_models`](https://github.com/rwightman/pytorch-image-models)
* ...

In [ ]:
from torchvision import models as M

ResNet - база. Возьмем восемнадцатый:

![resnet](https://velog.velcdn.com/images%2Fe_sin528%2Fpost%2Fe272c056-3dfa-4bb6-bfc9-b309d82df932%2FResNet18.png)

In [ ]:
resnet18 = M.resnet18(pretrained=True)

In [ ]:
resnet18;

In [ ]:
x = torch.randn(1, 3, 224, 224)

y = resnet18(x)
y.size()

Как использовать готовую модель?
* Заменить выходной слой на слой с нужным числом классов
  * `timm` умеет это прямо при инициализации
* Взять `feature_extractor` модели и навернуть сверху своих слоев
  * `timm` опять же позволяет это легко сделать
  
У нас особенный случай: ResNet18 уменьшает размер входного изображения в 64 раза, а у нас картинки 32х32. Как быть?

Например, можно взять и выдрать слои из модели и поместить в `Sequential`, приправив своими слоями сверху:

In [ ]:
cnn_finetuned = nn.Sequential(
    resnet18.conv1,
    resnet18.bn1,
    resnet18.relu,
    resnet18.maxpool,
    resnet18.layer1,
    resnet18.layer2,

    nn.AdaptiveAvgPool2d((1, 1)),
    nn.Flatten(),
    nn.Linear(128, 10)
)

In [ ]:
cnn_finetuned(x)

Теперь про обучение. Поскольку у нас есть частично обученные веса на входе (из ResNet) и полностью необученные на выходе (свои) веса, градиенты через конец сети могут быть очень шумными. Поэтому есть практика обучения только новых голов в течение пары эпох, а затем полное обучение:

In [ ]:
cnn_finetuned[0].weight.requires_grad, cnn_finetuned[-1].weight.requires_grad

Веса слоев можно заморозить ручками:

In [ ]:
for layer in cnn_finetuned:
    layer.requires_grad_(False)

In [ ]:
cnn_finetuned[0].weight.requires_grad, cnn_finetuned[-1].weight.requires_grad

In [ ]:
cnn_finetuned[-1].requires_grad_(True)
cnn_finetuned[-1].weight.requires_grad

Теперь к обучению:

In [ ]:
num_epochs = 32
batch_size = 128
lr = 3e-4

device = torch.device("cuda:7") if torch.cuda.is_available() else torch.device("cpu")
print(device)

In [ ]:
loss_fn = nn.CrossEntropyLoss()

Сначала 3 эпохи учим только последний слой:

In [ ]:
optimizer = torch.optim.Adam(cnn_finetuned[-1].parameters(), lr=lr)

In [ ]:
train_losses_finetuned, val_losses_finetuned, best_val_loss_finetuned, cnn_finetuned = run_experiment(
    cnn_finetuned, dataloader_aug_train, dataloader_val, loss_fn, optimizer, 3, device, "checkpoints_finetuned"
)

Теперь размораживаем всю сеть и учим целиком:

In [ ]:
for layer in cnn_finetuned:
    layer.requires_grad_(True)

optimizer = torch.optim.Adam(cnn_finetuned.parameters(), lr=lr)    

In [ ]:
train_losses_finetuned, val_losses_finetuned, best_val_loss_finetuned, cnn_finetuned = run_experiment(
    cnn_finetuned, dataloader_aug_train, dataloader_val, loss_fn, optimizer, num_epochs, device, "checkpoints_finetuned"
)

In [ ]:
plot_losses(train_losses_finetuned, val_losses_finetuned, title="cnn_finetuned")

In [ ]:
train_labels, train_probs, train_preds = collect_predictions(cnn_finetuned, dataloader_aug_train, device)

accuracy_train = accuracy_score(train_labels, train_preds)
accuracy_train

In [ ]:
val_labels, val_probs, val_preds = collect_predictions(cnn_finetuned, dataloader_val, device)

accuracy_val = accuracy_score(val_labels, val_preds)
accuracy_val

## 4. LR scheduling

Последняя на сегодня - работа с LR.
Из лекций вы могли запомнить, что варьирование LR при обучении (даже адаптивных методов) может достичь более высокого качества.

![lrs](https://i.stack.imgur.com/UHYMw.png)

`pytorch` предоставляет возможности и для этого.

### 4.1. Обновление по сигналу от метрик (ReduceLROnPlateau)

Можно изменять LR, основываясь на изменении целевой метрики.
Если, например, лосс давно не падает, можно уменьшить LR:

In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
cnn_aug = nn.Sequential(
    CNNBlock(3, 32),
    CNNBlock(32, 64),
    CNNBlock(64, 128),
    nn.AdaptiveAvgPool2d((1, 1)),
    nn.Flatten(),
    nn.Linear(128, 10)
)

In [ ]:
num_epochs = 16
batch_size = 128
lr = 1

device = torch.device("cuda:7") if torch.cuda.is_available() else torch.device("cpu")
print(device)

In [ ]:
loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(cnn_aug.parameters(), lr=lr)

scheduler = ReduceLROnPlateau(optimizer, mode="min", factor=0.1, patience=2, verbose=True)

In [ ]:
def run_epoch(stage, model, dataloader, loss_fn, optimizer, epoch, device):

    if stage == "train":
        model.train()
        torch.set_grad_enabled(True)
    else:
        torch.set_grad_enabled(False)
        model.eval()

    model = model.to(device)
    
    losses = []
    for batch in tqdm.tqdm(dataloader, total=len(dataloader), desc=f"epoch: {str(epoch).zfill(3)} | {stage:5}"):
        xs, ys_true = batch
                
        ys_pred = model(xs.to(device))
        loss = loss_fn(ys_pred, ys_true.to(device))

        if stage == "train":
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
                
        losses.append(loss.detach().cpu().item())
    
    if stage == "train":
        scheduler.step(np.mean(losses))

    return np.mean(losses)

In [ ]:
train_losses_aug, val_losses_aug, best_val_loss_aug, cnn_aug = run_experiment(
    cnn_aug, dataloader_aug_train, dataloader_val, loss_fn, optimizer, num_epochs, device, "checkpoints_aug"
)

### 4.2. Обновление каждую эпоху (StepLR)

In [ ]:
from torch.optim.lr_scheduler import StepLR

In [ ]:
cnn_aug = nn.Sequential(
    CNNBlock(3, 32),
    CNNBlock(32, 64),
    CNNBlock(64, 128),
    nn.AdaptiveAvgPool2d((1, 1)),
    nn.Flatten(),
    nn.Linear(128, 10)
)

In [ ]:
num_epochs = 8
batch_size = 128
lr = 3e-4

device = torch.device("cuda:7") if torch.cuda.is_available() else torch.device("cpu")
print(device)

In [ ]:
loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(cnn_aug.parameters(), lr=lr)

scheduler = StepLR(optimizer, step_size=2, gamma=0.1, verbose=True)

In [ ]:
def run_epoch(stage, model, dataloader, loss_fn, optimizer, epoch, device):

    if stage == "train":
        model.train()
        torch.set_grad_enabled(True)
    else:
        torch.set_grad_enabled(False)
        model.eval()

    model = model.to(device)
    
    losses = []
    for batch in tqdm.tqdm(dataloader, total=len(dataloader), desc=f"epoch: {str(epoch).zfill(3)} | {stage:5}"):
        xs, ys_true = batch
                
        ys_pred = model(xs.to(device))
        loss = loss_fn(ys_pred, ys_true.to(device))

        if stage == "train":
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
                
        losses.append(loss.detach().cpu().item())
    
    if stage == "train":
        scheduler.step()

    return np.mean(losses)

In [ ]:
train_losses_aug, val_losses_aug, best_val_loss_aug, cnn_aug = run_experiment(
    cnn_aug, dataloader_aug_train, dataloader_val, loss_fn, optimizer, num_epochs, device, "checkpoints_aug"
)

### 4.3. Обновление каждую итерацию (CosineAnnealingLR)

Есть разные техники изменения LR по заданному закону. 
Например, [CosineAnnealing](https://paperswithcode.com/method/cosine-annealing).

In [ ]:
from torch.optim.lr_scheduler import CosineAnnealingLR

In [ ]:
cnn_aug = nn.Sequential(
    CNNBlock(3, 32),
    CNNBlock(32, 64),
    CNNBlock(64, 128),
    nn.AdaptiveAvgPool2d((1, 1)),
    nn.Flatten(),
    nn.Linear(128, 10)
)

In [ ]:
num_epochs = 8
batch_size = 128
lr = 3e-4

device = torch.device("cuda:7") if torch.cuda.is_available() else torch.device("cpu")
print(device)

In [ ]:
loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(cnn_aug.parameters(), lr=lr)

scheduler = CosineAnnealingLR(optimizer, T_max=int(len(dataloader_aug_train) + 1) * num_epochs)

In [ ]:
def run_epoch(stage, model, dataloader, loss_fn, optimizer, epoch, device):

    if stage == "train":
        model.train()
        torch.set_grad_enabled(True)
        print("lr (epoch start):", scheduler.optimizer.param_groups[0]['lr'])
    else:
        torch.set_grad_enabled(False)
        model.eval()

    model = model.to(device)
    
    
    losses = []
    for batch in tqdm.tqdm(dataloader, total=len(dataloader), desc=f"epoch: {str(epoch).zfill(3)} | {stage:5}"):
        xs, ys_true = batch
                
        ys_pred = model(xs.to(device))
        loss = loss_fn(ys_pred, ys_true.to(device))

        if stage == "train":
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            scheduler.step()
            
        losses.append(loss.detach().cpu().item())
    
    if stage == "train":
        print("lr (epoch end):", scheduler.optimizer.param_groups[0]['lr'])

    return np.mean(losses)

In [ ]:
train_losses_aug, val_losses_aug, best_val_loss_aug, cnn_aug = run_experiment(
    cnn_aug, dataloader_aug_train, dataloader_val, loss_fn, optimizer, num_epochs, device, "checkpoints_aug"
)

## Итоги

* Научились добавлять аугментации к обучению
* Познакомились с методом использования предобученных моделей
* Посмотрели на работу с LR scheduling в pytorch.

Рекомендуется (в который раз) почитать пост от любимого нашего Andrej Karpathy [A Recipe for Training Neural Networks](https://karpathy.github.io/2019/04/25/recipe/) и обзорную статью (не самую новую, но все же) по трюкам для обучения моделей [Bag of Tricks for Image Classification with Convolutional Neural Networks](https://arxiv.org/pdf/1812.01187v2.pdf).

Впереди ждет соревнование по классификации картинок на Kaggle, где вы сможете применить все полученные (и неполученные %)) знания на практике!